<div style="background-color: orange">
<a id='TableOfContents'></a>
    <b><u><i><h1 style='text-align: center'>
        Table of Contents
    </h1></i></u></b>
<li><a href='#imports'>Imports</a></li>
<li><a href='#wrangle'>Wrangle Data</a></li>
<li><a href='#modelsdict'>Models Dictionary Creation</a></li>
<li><a href='#baseline'>Baseline Creation</a></li>
<li><a href='#dtc'>Decision Tree Classifier</a></li>
<li><a href='#rfc'>Random Forest Classifiier</a></li>
<li><a href='#lr'>Linear Regression</a></li>
<li><a href='#misc'>Miscellaneous</a></li>

<div style='background-color: orange'>
<a id='imports'></a>
    <b><u><i><h1 style='text-align: center'>
        Imports
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [1]:
# Vectorization and dataframing
import numpy as np
import pandas as pd

# Visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Classification Models
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.linear_model import LinearRegression as lr

# Model Metrics
from sklearn.metrics import classification_report, accuracy_score, recall_score

#.py Files
import wrangle as w

<div style='background-color: orange'>
<a id='wrangle'></a>
    <b><u><i><h1 style='text-align: center'>
        Wrangle Data
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [2]:
# Wrangle and split the mass_shooters data for proper
# modeling purposes
train, validate, test = w.wrangle_mass_shooters()

In [3]:
# Verify split data
train.shape, validate.shape, test.shape

((131, 254), (38, 254), (19, 254))

<div style='background-color: orange'>
<a id='modelsdict'></a>
    <b><u><i><h1 style='text-align: center'>
        Models Dictionary Creation
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [22]:
# Create a dictionary that holds all the models for 
# easier comparison purposes...
models_dict = {
    'model_name' : [],
    'model_type' : [],
    'model_descriptor' : [],
    'accuracy_train' : [],
    'accuracy_val' : [],
    'accuracy_diff' : [],
    'recall_variable' : [],
    'recall_train' : [],
    'recall_val' : [],
    'recall_diff' : []
}

<div style='background-color: orange'>
<a id='baseline'></a>
    <b><u><i><h1 style='text-align: center'>
        Baseline Creation
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

In [24]:
# Target value is 'shooter_volatility', thus create
# baseline with mode value ('Low Volatility')
baseline_train = round((train.shooter_volatility == train.shooter_volatility.mode()[0]).sum() / train.shape[0], 3)
baseline_val = round((validate.shooter_volatility == validate.shooter_volatility.mode()[0]).sum() / validate.shape[0], 3)

# Append to 'models_dict'
models_dict['model_name'].append('Baseline')
models_dict['model_type'].append('Baseline')
models_dict['model_descriptor'].append('Mode = Low Volatility')
models_dict['accuracy_train'].append(baseline_train)
models_dict['accuracy_val'].append(baseline_val)
models_dict['accuracy_diff'].append(baseline_val - baseline_train)
models_dict['recall_variable'].append('High Volatility')
models_dict['recall_train'].append(0.000)
models_dict['recall_val'].append(0.000)
models_dict['recall_diff'].append(0.000)

In [25]:
# Check models_dict via pd.DataFrame
pd.DataFrame(models_dict)

,model_name,model_type,model_descriptor,accuracy_train,accuracy_val,accuracy_diff,recall_variable,recall_train,recall_val,recall_diff
0,Baseline,Baseline,Mode = Low Volatility,0.634,0.632,-0.002,High Volatility,0.0,0.0,0.0


<div style='background-color: orange'>
<a id='dtc'></a>
    <b><u><i><h1 style='text-align: center'>
        Decision Tree Classifier
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<div style='background-color: orange'>
<a id='rfc'></a>
    <b><u><i><h1 style='text-align: center'>
        Random Forest Classifier
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<div style='background-color: orange'>
<a id='lr'></a>
    <b><u><i><h1 style='text-align: center'>
        Linear Regression
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>

<div style='background-color: orange'>
<a id='misc'></a>
    <b><u><i><h1 style='text-align: center'>
        Miscellaneous
    </h1></i></u></b>
<li><a href='#TableOfContents'>Table of Contents</a></li>